In [2]:
import random
import math
import numpy as np
import pandas as pd
import hvplot.pandas
from operator import xor
from functools import reduce
from itertools import islice
from gps_coders import GPS_L1_PRNS, GPS_L1_coder

In [3]:
print(list(islice(GPS_L1_coder(1), 10)))

[1, 1, 0, 0, 1, 0, 0, 0, 0, 0]


In [4]:
def make_rprn(seed, length):
    
    g = np.random.default_rng()
    return g.integers(0,2, length)*2-1

In [5]:
CODELEN = 1023

In [6]:
#prns = [make_rprn(i, CODELEN) for i in range(32)]
prn_codes = { p: np.array(list(islice(GPS_L1_coder(p), CODELEN))) for p in GPS_L1_PRNS}

In [7]:
prn_codes[1][:10]

array([1, 1, 0, 0, 1, 0, 0, 0, 0, 0])

In [20]:
chip_rate = 1023_000 # chips/s
chip_duration = 1 / chip_rate
print(f'chip duration: {chip_duration}s')

chip duration: 9.775171065493646e-07s


In [9]:
def chip(t, prn):
    return prn[math.floor(t/chip_duration) % len(prn)]

In [10]:
idx = 11
cs = [ prn_codes[idx][i] for i in range(10)]
ds = [ chip(t, prn_codes[idx]) for t in (i * chip_duration for i in range(10))]

In [11]:
print(cs)
print(ds)

[np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0)]
[np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0)]


In [21]:
nano_sec_samples = range(0, 10000)

In [22]:
ds = pd.DataFrame(
    data= {
        prn: [chip(t/1E9, prn_codes[prn]) for t in nano_sec_samples]
        for prn in GPS_L1_PRNS
    },
    index=nano_sec_samples
)

In [25]:
import hvplot.pandas  # noqa

pd.options.plotting.backend = 'holoviews'

from hvplot import hvPlot
hvplot.extension('bokeh')


In [27]:
plot = hvPlot(ds)
plot(y=['1','2','3'])

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [46]:
def correlate(prn0, t0, prn1, t1, start, end, numsamples):
    return sum(abs(chip(t0+d, prn0)+chip(t1+d,prn1)) for d in np.linspace(start, end, num=numsamples))/(2*numsamples)


In [47]:
print(CODELEN*chip_duration)

0.001


In [52]:
N = 0
d = 1
offsets = [o*d*chip_duration for o in range(N, N+10)]
for offset in offsets:
    print(f'offset: {offset}, correlation: {correlate(prn_codes[1], 0, prn_codes[1], 0+offset, 0, 1, 10):5.3}')

offset: 0.0, correlation:   0.5
offset: 9.775171065493646e-07, correlation:   0.5
offset: 1.955034213098729e-06, correlation:  0.55
offset: 2.9325513196480935e-06, correlation:  0.55
offset: 3.910068426197458e-06, correlation:   0.6
offset: 4.887585532746823e-06, correlation:  0.45
offset: 5.865102639296187e-06, correlation:  0.45
offset: 6.842619745845552e-06, correlation:  0.45
offset: 7.820136852394917e-06, correlation:  0.35
offset: 8.79765395894428e-06, correlation:  0.45


There is a small difference if `1001*CODELEN` is replaced by `1000*CODELEN`. This is probable a rounding error in float arithmetic.

TypeError: 'float' object cannot be interpreted as an integer

In [16]:
from bokeh.sampledata.penguins import data as df

In [17]:
explorer = df.hvplot.explorer(x='bill_length_mm', y='bill_depth_mm', by=['species'])
explorer

hvDataFrameExplorer(advanced=Advanced(explorer=..., name='Advanced03754', opts=None), axes=Axes(explorer=..., height=None, legend='bottom_right', logx=False, logy=False, name='Axes03748', responsive=True, shared_axes=True, width=None, xlim=None, ylim=None), by=['species'], code="df.hvplot(\n    by=['species'],\n    kind='scatter',\n    x='bill_length_mm',\n    y=['bill_depth_mm'],\n    legend='bottom_right',\n    widget_location='bottom',\n)", colormapping=Colormapping(clim=None, cmap='kbc_r', cnorm='linear', color=None, colorbar=None, explorer=..., name='Colormapping03749', rescale_discrete_levels=True, symmetric=False), geographic=Geographic(crs=None, crs_kwargs={}, explorer=..., feature_scale='110m', features=None, geo=False, global_extent=None, name='Geographic03751', project=False, projection=None, projection_kwargs={}, tiles=None), groupby=[], kind='scatter', labels=Labels(clabel='', explorer=..., fontscale=1, name='Labels03750', rot=0, title='', xlabel='', ylabel=''), name='hvDataFrameExplorer03726', operations=Operations(aggregator=None, datashade=False, dynspread=False, explorer=..., name='Operations03752', rasterize=False, x_sampling=None, y_sampling=None), statusbar=StatusBar(live_update=True, name='StatusBar03741'), style=Style(alpha=1, explorer=..., name='Style03753'), x='bill_length_mm', y='bill_depth_mm', y_multi=['bill_depth_mm'], z=None)